In [54]:
pd.read_csv("../output/ddos_1s/combined_metrics.csv")

,model,f1,precision,recall,accuracy,average_precision,log_loss
0,RNNModel,0.952733,0.921429,0.986239,0.940075,0.917175,0.155102
1,LSTMModel,0.958269,0.968750,0.948012,0.949438,0.950222,0.124808
2,XGBoostModel,0.974610,0.970645,0.978607,0.970310,0.962336,0.097744
3,LogisticRegressionModel,0.863058,0.839004,0.888531,0.835815,0.810387,0.432541
4,RandomForestModel,0.988940,0.992707,0.985202,0.987169,0.986633,0.045054
5,GRUModel,0.949358,0.938714,0.960245,0.937266,0.925740,0.139630


In [43]:
model = BayesSearchCV(
                LogisticRegression(),
                {
                    'C' : (1e-3, 1e+5, 'log-uniform'),
                    'solver' :['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],

                },
                n_iter=100,
                cv=3,
                scoring='f1',
                n_jobs=-1,
            )

In [45]:
isinstance(model, BayesSearchCV)

True

In [36]:
a = np.array([1,2,3])

In [22]:
from pathlib import Path

In [23]:
a = Path('../data/aggregated_binetflows/ddos_1s.csv')

In [7]:
data = pd.read_csv('../data/aggregated_binetflows/ddos_1s.csv')

In [12]:
label_col = 'label'

In [20]:
data.fillna(0,axis=None, inplace=True)
data[label_col] = data['n_conn'] - data['n_background'] - data['n_normal']
data[label_col] = data[label_col].apply(lambda x: 1 if x > 0 else 0)

In [21]:
data.label.value_counts()

1    20850
0    14740
Name: label, dtype: int64

In [11]:
data.columns

Index(['m_duration', 'n_background', 'n_conn', 'n_dst_ip_a', 'n_dst_ip_b',
       'n_dst_ip_c', 'n_dst_ip_na', 'n_dst_port<1024', 'n_dst_port>=1024',
       'n_icmp', 'n_normal', 'n_src_ip_a', 'n_src_ip_b', 'n_src_ip_c',
       'n_src_ip_na', 'n_src_port<1024', 'n_src_port>=1024', 'n_tcp', 'n_udp',
       's_dst_ip_a', 's_dst_ip_b', 's_dst_ip_c', 's_dst_ip_d',
       's_dst_port<1024', 's_dst_port>=1024', 's_dstip', 's_duration',
       's_packets', 's_src_ip_a', 's_src_ip_b', 's_src_ip_c', 's_src_ip_d',
       's_src_port<1024', 's_src_port>=1024', 's_srcbytes', 's_srcip',
       's_state', 's_totbytes', 'sd_duration', 'sd_packets', 'sd_srcbytes',
       'sd_totbytes'],
      dtype='object')

In [6]:
import pandas as pd

/data/saed/env/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
/data/saed/env/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)


In [4]:
from models.random_forest import RandomForestModel

In [5]:
RandomForestModel()

'RandomForestModel'

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import click
import logging as LOGGER


LOGGER.basicConfig(format="%(asctime)s %(levelname)s %(message)s", level=LOGGER.INFO)


@click.command()
@click.argument("aggregated_data_file", type=click.Path(exists=True))
@click.argument("label_column")
@click.argument("output_directory")
@click.option("--test_set_size", default=0.3, type=click.FLOAT)
@click.option("--random_seed", default=2018, type=click.INT)
@click.option(
    "--model_list",
    multiple=True,
    default=["random_forest", "xgboost", "logistic_regression"],
    help="List of model classes to train",
)
def train_models(
    data_file, label_column, output_directory, test_set_size, random_seed, model_list
):
    """ Trains multiple machine learning models for a specific datatable.
    """
    LOGGER.info(f"Reading {data_file} ...")
    data = pd.read_csv(data_file)
#     data = preprocess_data(data)

    x_tr, x_te, y_tr, y_te = split_data(data, test_set_size, random_seed)

    if all([not do_random_forest, not do_xgboost, not do_lr]):
        raise ValueError(
            "Specify models to train with one or more of these flags; --rf, --xgb, --lr"
        )

    mapper = {"random_forst": object, "xgboost": object, "logistic_regression": object}

    for model_name in model_list:
        LOGGER.info(f"Instantiating {model_name} ...")

        if model_name not in mapper:
            LOGGER.warning(f"Model {model_name} is unknown. Skipping it!")
            continue

        model = mapper[model_name]()
        LOGGER.info(f"Fitting {model_name} to the train set ...")
        model.fit(x_tr, y_tr)
        LOGGER.info(f"Evaluating {model_name} on the test set ...")
        model.evaluate(x_te, y_te)

    LOGGER.info("Done training all the models.")